# ห้องปฏิบัติการเสริม: การประเมินและเลือกโมเดล

การหาปริมาณประสิทธิภาพของอัลกอริทึมการเรียนรู้และการเปรียบเทียบโมเดลต่างๆ เป็นหนึ่งในงานทั่วไปเมื่อนำการเรียนรู้ของเครื่องไปใช้ในแอปพลิเคชันในโลกแห่งความจริง ในห้องปฏิบัติการนี้ คุณจะฝึกทำสิ่งเหล่านี้โดยใช้เคล็ดลับที่แบ่งปันในชั้นเรียน โดยเฉพาะอย่างยิ่ง คุณจะ:

* แยกชุดข้อมูลออกเป็นชุดฝึก ชุดตรวจสอบข้าม และชุดทดสอบ
* ประเมินโมเดลการถดถอยและการจำแนกประเภท
* เพิ่มคุณลักษณะพหุนามเพื่อปรับปรุงประสิทธิภาพของโมเดลการถดถอยเชิงเส้น
* เปรียบเทียบสถาปัตยกรรมเครือข่ายประสาทเทียมหลายแบบ

ห้องปฏิบัติการนี้จะช่วยให้คุณคุ้นเคยกับโค้ดที่คุณจะเห็นในการมอบหมายการเขียนโปรแกรมในสัปดาห์นี้ มาเริ่มกัน!

## การนำเข้าและการตั้งค่าห้องปฏิบัติการ

ขั้นแรก คุณจะนำเข้าแพ็คเกจที่จำเป็นสำหรับงานในห้องปฏิบัติการนี้ เรายังได้รวมคำสั่งบางอย่างเพื่อทำให้ผลลัพธ์ในภายหลังอ่านได้ง่ายขึ้นโดยการลดความยืดยาวและระงับการเตือนที่ไม่สำคัญ

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/utils.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/data/data_w3_ex1.csv'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/data/data_w3_ex2.csv'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('utils.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
data_path = Path('data')
if not data_path.is_dir():
  data_path.mkdir(parents=True, exist_ok=True)
with open('data/data_w3_ex1.csv', 'wb') as f:
  f.write(response.content)
with open('data/data_w3_ex2.csv', 'wb') as f:
  f.write(response.content)




In [ ]:
# for array computations and loading data
import numpy as np

# for building linear regression models and preparing data
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# for building and training neural networks
import tensorflow as tf

# custom functions
import utils

# reduce display precision on numpy arrays
np.set_printoptions(precision=2)

# suppress warnings
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

## การถดถอย (Regression)

อันดับแรก คุณจะได้รับมอบหมายให้พัฒนาโมเดลสำหรับปัญหาการถดถอย คุณได้รับชุดข้อมูลด้านล่างซึ่งประกอบด้วยตัวอย่าง 50 ตัวอย่างของคุณลักษณะอินพุต `x` และเป้าหมายที่สอดคล้องกัน `y`.

In [ ]:
# Load the dataset from the text file
data = np.loadtxt('./data/data_w3_ex1.csv', delimiter=',')

# Split the inputs and outputs into separate arrays
x = data[:,0]
y = data[:,1]

# Convert 1-D arrays into 2-D because the commands later will require it
x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)

print(f"the shape of the inputs x is: {x.shape}")
print(f"the shape of the targets y is: {y.shape}")

คุณสามารถพล็อตชุดข้อมูลเพื่อดูพฤติกรรมของเป้าหมายสัมพันธ์กับอินพุต ในกรณีที่คุณต้องการตรวจสอบโค้ด คุณสามารถดูฟังก์ชัน `plot_dataset()` ได้ในไฟล์ `utils.py` นอกเหนือจากสมุดบันทึกนี้.

In [ ]:
# Plot the entire dataset
utils.plot_dataset(x=x, y=y, title="input vs. target")

## แบ่งชุดข้อมูลออกเป็นชุดฝึก ชุดตรวจสอบข้าม และชุดทดสอบ

ในห้องปฏิบัติการก่อนหน้านี้ คุณอาจใช้ชุดข้อมูลทั้งหมดในการฝึกโมเดลของคุณ อย่างไรก็ตาม ในทางปฏิบัติแล้ว ควรแยกส่วนหนึ่งของข้อมูลของคุณเพื่อวัดว่าโมเดลของคุณสามารถสรุปข้อมูลใหม่ได้ดีแค่ไหน ซึ่งจะช่วยให้คุณทราบว่าโมเดลได้เรียนรู้มากเกินไปกับชุดฝึกของคุณหรือไม่

ตามที่กล่าวไว้ในบรรยาย เป็นเรื่องปกติที่จะแบ่งข้อมูลของคุณออกเป็นสามส่วน:

* ***training set*** - ใช้ในการฝึกโมเดล
* ***cross validation set (cross validation set หรือ validation, development, หรือ dev set)*** - ใช้ในการประเมินการกำหนดค่าโมเดลที่แตกต่างกันที่คุณกำลังเลือก ตัวอย่างเช่น คุณสามารถใช้สิ่งนี้เพื่อตัดสินใจว่าจะเพิ่มคุณลักษณะพหุนามใดลงในชุดข้อมูลของคุณ
* ***test set*** - ใช้เพื่อให้การประมาณค่าที่เป็นธรรมเกี่ยวกับประสิทธิภาพของโมเดลที่คุณเลือกกับตัวอย่างใหม่ สิ่งนี้ไม่ควรใช้ในการตัดสินใจขณะที่คุณยังคงพัฒนาโมเดล

Scikit-learn มีฟังก์ชัน [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) เพื่อแบ่งข้อมูลของคุณออกเป็นส่วนต่างๆ ดังที่กล่าวมาข้างต้น ในเซลล์โค้ดด้านล่าง คุณจะแบ่งชุดข้อมูลทั้งหมดออกเป็น 60% สำหรับการฝึก 20% สำหรับการตรวจสอบข้าม และ 20% สำหรับการทดสอบ

In [ ]:
# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary variables
del x_, y_

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

คุณสามารถพล็อตชุดข้อมูลอีกครั้งด้านล่างเพื่อดูว่าจุดใดถูกใช้เป็นข้อมูลการฝึก การตรวจสอบข้ามหรือข้อมูลทดสอบ

In [ ]:
utils.plot_train_cv_test(x_train, y_train, x_cv, y_cv, x_test, y_test, title="input vs. target")

## ปรับโมเดลเชิงเส้น

ตอนนี้คุณได้แยกข้อมูลแล้ว หนึ่งในสิ่งแรกที่คุณสามารถลองทำได้คือการปรับโมเดลเชิงเส้น คุณจะทำเช่นนั้นในส่วนต่อไปด้านล่าง

### การปรับขนาดฟีเจอร์

ในหลักสูตรก่อนหน้านี้ของความเชี่ยวชาญนี้ คุณได้เห็นว่า โดยปกติแล้วการปรับขนาดฟีเจอร์ (feature scaling) เป็นสิ่งที่ดีที่ควรทำเพื่อช่วยให้โมเดลของคุณบรรจบได้เร็วขึ้น โดยเฉพาะอย่างยิ่งหากฟีเจอร์อินพุตของคุณมีช่วงของค่าที่ต่างกันอย่างมาก ในบทฝึกปฏิบัติในภายหลังของบทเรียนนี้ คุณจะเพิ่มเทอมพหุนามเข้าไป ดังนั้นฟีเจอร์อินพุตของคุณจะมีช่วงที่ต่างกัน ตัวอย่างเช่น $x$ จะมีค่าตั้งแต่ประมาณ 1600 ถึง 3600 ในขณะที่ $x^2$  จะมีค่าตั้งแต่ 2.56 ล้านถึง 12.96 ล้าน

สำหรับโมเดลแรกนี้ คุณจะใช้เฉพาะ x เท่านั้น แต่การฝึกฝนการปรับขนาดฟีเจอร์ตั้งแต่ตอนนี้เป็นสิ่งที่ดี เพื่อที่คุณจะสามารถนำไปใช้ในภายหลังได้ ในการนั้น คุณจะใช้คลาส  [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) จาก scikit-learn ซึ่งคำนวณคะแนน Z ของข้อมูลอินพุตของคุณ เพื่อเป็นการทบทวน คะแนน Z จะได้จากสมการ:

$$ z = \frac{x - \mu}{\sigma} $$

โดยที่  $\mu$ คือค่าเฉลี่ยของค่าลักษณะและ $\sigma$ คือส่วนเบี่ยงเบนมาตรฐาน โค้ดด้านล่างแสดงวิธีการเตรียมชุดฝึกอบรมโดยใช้คลาสที่กล่าวถึง คุณสามารถพล็อตผลลัพธ์อีกครั้งเพื่อตรวจสอบว่ายังคงเป็นไปตามรูปแบบเดิมหรือไม่ กราฟใหม่ควรมีช่วงของค่าที่ลดลงสำหรับ `x`.

In [ ]:
# Initialize the class
scaler_linear = StandardScaler()

# Compute the mean and standard deviation of the training set then transform it
X_train_scaled = scaler_linear.fit_transform(x_train)

print(f"Computed mean of the training set: {scaler_linear.mean_.squeeze():.2f}")
print(f"Computed standard deviation of the training set: {scaler_linear.scale_.squeeze():.2f}")

# Plot the results
utils.plot_dataset(x=X_train_scaled, y=y_train, title="scaled input vs. target")

### ฝึกโมเดล
ต่อไป คุณจะสร้างและฝึกโมเดลการถดถอย สำหรับห้องปฏิบัติการนี้ คุณจะใช้คลาส  [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) แต่โปรดทราบว่ามีตัวถดถอยเชิงเส้นอื่น ๆ  [linear regressors](https://scikit-learn.org/stable/modules/classes.html#classical-linear-regressors) ที่คุณสามารถใช้ได้เช่นกัน

In [ ]:
# Initialize the class
linear_model = LinearRegression()

# Train the model
linear_model.fit(X_train_scaled, y_train )

### การประเมินประสิทธิภาพของโมเดล
เพื่อประเมินประสิทธิภาพของโมเดลของคุณ เราจะวัดค่าความผิดพลาดสำหรับชุดข้อมูลฝึกอบรมและชุดข้อมูลตรวจสอบข้าม (cross validation)

สำหรับความผิดพลาดของการฝึกอบรม ทบทวนสูตรสำหรับการคำนวณค่าความผิดพลาดกำลังสองเฉลี่ย (mean squared error: MSE)


$$J_{train}(\vec{w}, b) = \frac{1}{2m_{train}}\left[\sum_{i=1}^{m_{train}}(f_{\vec{w},b}(\vec{x}_{train}^{(i)}) - y_{train}^{(i)})^2\right]$$

Scikit-learn ยังมีฟังก์ชัน [`mean_squared_error()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) ที่คุณสามารถใช้ได้ อย่างไรก็ตาม โปรดทราบว่า [as per the documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)การใช้งานของ scikit-learn จะหารด้วย  `m` เท่านั้น ไม่ใช่ `2*m`, โดยที่  `m` คือจำนวนตัวอย่าง ดังที่กล่าวไว้ใน Course 1 ของความเชี่ยวชาญนี้ ( บทบรรยายฟังก์ชันต้นทุน) การหารด้วย `2m`  เป็นรูปแบบที่เราจะปฏิบัติตาม แต่การคำนวณจะยังคงทำงานได้ไม่ว่าคุณจะรวมไว้หรือไม่ ดังนั้น เพื่อให้ตรงกับสมการด้านบน คุณสามารถใช้ฟังก์ชัน scikit-learn แล้วหารด้วย 2 ดังที่แสดงด้านล่าง นอกจากนี้ เรายังรวมการวนซ้ำแบบฟังก์ชันไว้เพื่อให้คุณตรวจสอบได้ว่ามันเท่ากัน

อีกสิ่งที่ต้องคำนึงถึง: เนื่องจากคุณได้ฝึกอบรมโมเดลบนค่าที่ปรับขนาดแล้ว (เช่น ใช้คะแนนมาตรฐาน) คุณควรป้อนข้อมูลชุดข้อมูลฝึกอบรมที่ปรับขนาดแล้วแทนค่าดิบด้วย


In [ ]:
# Feed the scaled training set and get the predictions
yhat = linear_model.predict(X_train_scaled)

# Use scikit-learn's utility function and divide by 2
print(f"training MSE (using sklearn function): {mean_squared_error(y_train, yhat) / 2}")

# for-loop implementation
total_squared_error = 0

for i in range(len(yhat)):
    squared_error_i  = (yhat[i] - y_train[i])**2
    total_squared_error += squared_error_i                                              

mse = total_squared_error / (2*len(yhat))

print(f"training MSE (for-loop implementation): {mse.squeeze()}")

คุณสามารถคำนวณ MSE สำหรับชุดการตรวจสอบแบบไขว้ได้ด้วยสมการเดียวกัน:
$$J_{cv}(\vec{w}, b) = \frac{1}{2m_{cv}}\left[\sum_{i=1}^{m_{cv}}(f_{\vec{w},b}(\vec{x}_{cv}^{(i)}) - y_{cv}^{(i)})^2\right]$$

เช่นเดียวกับชุดการฝึก คุณจะต้องปรับขนาดชุดการตรวจสอบแบบไขว้ด้วย สิ่งสำคัญที่ต้องทราบเมื่อใช้ z-score คือคุณต้องใช้ค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานของชุดการฝึกเมื่อปรับขนาดชุดการตรวจสอบแบบไขว้ สิ่งนี้เพื่อให้แน่ใจว่าคุณสมบัติอินพุตของคุณถูกแปลงตามที่คาดหวังโดยโมเดล วิธีหนึ่งในการได้รับสัญชาตญาณคือด้วยสถานการณ์นี้:

* สมมติว่าชุดการฝึกของคุณมีคุณสมบัติอินพุตเท่ากับ `500`  ซึ่งถูกปรับขนาดลงเป็น `0.5` โดยใช้ z-score
* หลังจากการฝึก โมเดลของคุณสามารถแมปอินพุตที่ปรับขนาดแล้ว `x=0.5` ไปยังเอาต์พุตเป้าหมาย `y=300` ได้อย่างแม่นยำ 
* ตอนนี้สมมติว่าคุณใช้งานโมเดลนี้และหนึ่งในผู้ใช้ของคุณป้อนตัวอย่างเท่ากับ  `500`. 
* หากคุณรับ z-score ของตัวอย่างอินพุตนี้โดยใช้ค่าอื่นใดของค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐาน อาจจะไม่ได้ปรับขนาดเป็น `0.5` และโมเดลของคุณอาจทำนายผิด (เช่น ไม่เท่ากับ `y=300`). 

คุณจะปรับขนาดชุดการตรวจสอบแบบไขว้ด้านล่างโดยใช้  `StandardScaler` เดียวกันที่คุณใช้ก่อนหน้านี้ แต่เรียก [`transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.transform) แทนที่จะใช้ [`fit_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.fit_transform).

In [ ]:
# Scale the cross validation set using the mean and standard deviation of the training set
X_cv_scaled = scaler_linear.transform(x_cv)

print(f"Mean used to scale the CV set: {scaler_linear.mean_.squeeze():.2f}")
print(f"Standard deviation used to scale the CV set: {scaler_linear.scale_.squeeze():.2f}")

# Feed the scaled cross validation set
yhat = linear_model.predict(X_cv_scaled)

# Use scikit-learn's utility function and divide by 2
print(f"Cross validation MSE: {mean_squared_error(y_cv, yhat) / 2}")

## Adding Polynomial Features (การเพิ่มคุณสมบัติพหุนาม)

จากกราฟก่อนหน้านี้ คุณอาจสังเกตเห็นว่าเป้าหมาย `y` เพิ่มขึ้นอย่างรวดเร็วขึ้นที่ค่า `x` ที่ต่ำกว่าเมื่อเทียบกับค่าที่สูงกว่า เส้นตรงอาจไม่ใช่ทางเลือกที่ดีที่สุดเพราะเป้าหมาย `y` ดูเหมือนจะแบนราบเมื่อ `x`  เพิ่มขึ้น ตอนนี้คุณมีค่าเหล่านี้ของ MSE การฝึกและการตรวจสอบข้ามจากโมเดลเชิงเส้น คุณสามารถลองเพิ่มคุณสมบัติพหุนามเพื่อดูว่าคุณสามารถได้ผลลัพธ์ที่ดีขึ้นหรือไม่ โค้ดส่วนใหญ่จะเหมือนกัน แต่จะมีขั้นตอนการเตรียมข้อมูลเพิ่มเติมเล็กน้อย มาดูด้านล่างกัน

### สร้างฟีเจอร์เพิ่มเติม

อันดับแรก คุณจะสร้างฟีเจอร์พหุนามจากชุดข้อมูลฝึกของคุณ โค้ดด้านล่างแสดงวิธีการทำโดยใช้คลาส [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)  มันจะสร้างฟีเจอร์อินพุตใหม่ซึ่งมีค่ากำลังสองของอินพุต `x` (เช่น degree=2)

In [ ]:
# Instantiate the class to make polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)

# Compute the number of features and transform the training set
X_train_mapped = poly.fit_transform(x_train)

# Preview the first 5 elements of the new training set. Left column is `x` and right column is `x^2`
# Note: The `e+<number>` in the output denotes how many places the decimal point should 
# be moved. For example, `3.24e+03` is equal to `3240`
print(X_train_mapped[:5])

จากนั้นคุณจะปรับขนาดอินพุตตามเดิมเพื่อลดช่วงของค่า

In [ ]:
# Instantiate the class
scaler_poly = StandardScaler()

# Compute the mean and standard deviation of the training set then transform it
X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)

# Preview the first 5 elements of the scaled training set.
print(X_train_mapped_scaled[:5])

จากนั้นคุณสามารถดำเนินการฝึกโมเดล หลังจากนั้น คุณจะวัดประสิทธิภาพของโมเดลกับชุดการตรวจสอบข้ามเหมือนก่อน คุณควรตรวจสอบให้แน่ใจว่าได้ดำเนินการแปลงข้อมูลเหมือนกับที่คุณทำในชุดการฝึก คุณจะเพิ่มจำนวนคุณลักษณะพหุนามเท่ากันจากนั้นปรับขนาดช่วงของค่า

In [ ]:
# Initialize the class
model = LinearRegression()

# Train the model
model.fit(X_train_mapped_scaled, y_train )

# Compute the training MSE
yhat = model.predict(X_train_mapped_scaled)
print(f"Training MSE: {mean_squared_error(y_train, yhat) / 2}")

# Add the polynomial features to the cross validation set
X_cv_mapped = poly.transform(x_cv)

# Scale the cross validation set using the mean and standard deviation of the training set
X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

# Compute the cross validation MSE
yhat = model.predict(X_cv_mapped_scaled)
print(f"Cross validation MSE: {mean_squared_error(y_cv, yhat) / 2}")

คุณจะสังเกตเห็นว่า MSE มีความดีขึ้นอย่างมากสำหรับทั้งชุดฝึกและชุดตรวจสอบข้ามเมื่อคุณเพิ่มพหุนามลำดับที่ 2 คุณอาจต้องการแนะนำพหุนามเพิ่มเติมและดูว่าอันไหนให้ประสิทธิภาพที่ดีที่สุด ดังที่แสดงในคลาส คุณสามารถมี 10 โมเดลที่แตกต่างกันเช่นนี้:

<img src='images/C2_W3_poly.png' width=50%>

คุณสามารถสร้างลูปที่ประกอบด้วยขั้นตอนทั้งหมดในเซลล์โค้ดก่อนหน้านี้ นี่คือการใช้งานหนึ่งที่เพิ่มฟีเจอร์พหุนามสูงสุดถึงระดับ 10 เราจะพล็อตมันในตอนท้ายเพื่อให้เปรียบเทียบผลลัพธ์สำหรับแต่ละโมเดลได้ง่ายขึ้น

In [ ]:
# Initialize lists to save the errors, models, and feature transforms
train_mses = []
cv_mses = []
models = []
polys = []
scalers = []

# Loop over 10 times. Each adding one more degree of polynomial higher than the last.
for degree in range(1,11):
    
    # Add polynomial features to the training set
    poly = PolynomialFeatures(degree, include_bias=False)
    X_train_mapped = poly.fit_transform(x_train)
    polys.append(poly)
    
    # Scale the training set
    scaler_poly = StandardScaler()
    X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)
    scalers.append(scaler_poly)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train_mapped_scaled, y_train )
    models.append(model)
    
    # Compute the training MSE
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    train_mses.append(train_mse)
    
    # Add polynomial features and scale the cross validation set
    X_cv_mapped = poly.transform(x_cv)
    X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)
    
    # Compute the cross validation MSE
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    cv_mses.append(cv_mse)
    
# Plot the results
degrees=range(1,11)
utils.plot_train_cv_mses(degrees, train_mses, cv_mses, title="degree of polynomial vs. train and CV MSEs")

### การแปลเป็นภาษาไทย
"การเลือกโมเดลที่ดีที่สุด

เมื่อเลือกโมเดล คุณควรเลือกโมเดลที่ทำงานได้ดีทั้งในชุดฝึกและชุดตรวจสอบข้ามข้อมูล ซึ่งหมายความว่าโมเดลสามารถเรียนรู้รูปแบบจากชุดฝึกของคุณได้โดยไม่เกิดการโอเวอร์ฟิต หากคุณใช้ค่าเริ่มต้นในห้องปฏิบัติการนี้ คุณจะสังเกตเห็นการลดลงอย่างมากของข้อผิดพลาดในการตรวจสอบข้ามข้อมูลจากโมเดลที่มีดีกรี=1 ไปเป็นดีกรี=2 ตามด้วยเส้นที่ค่อนข้างราบจนถึงดีกรี=5 อย่างไรก็ตาม หลังจากนั้น ข้อผิดพลาดในการตรวจสอบข้ามข้อมูลโดยทั่วไปจะแย่ลงเมื่อคุณเพิ่มคุณลักษณะพหุนามมากขึ้น ด้วยเหตุนี้ คุณสามารถตัดสินใจใช้โมเดลที่มี `cv_mse` ต่ำที่สุดเป็นโมเดลที่เหมาะสมที่สุดสำหรับแอปพลิเคชันของคุณ

In [ ]:
# Get the model with the lowest CV MSE (add 1 because list indices start at 0)
# This also corresponds to the degree of the polynomial added
degree = np.argmin(cv_mses) + 1
print(f"Lowest CV MSE is found in the model with degree={degree}")

คุณสามารถเผยแพร่ข้อผิดพลาดทั่วไปโดยการคำนวณ MSE ของชุดทดสอบ ตามปกติ คุณควรแปลงข้อมูลนี้ในลักษณะเดียวกับที่คุณทำกับชุดฝึกและชุดตรวจสอบข้าม

In [ ]:
# Add polynomial features to the test set
X_test_mapped = polys[degree-1].transform(x_test)

# Scale the test set
X_test_mapped_scaled = scalers[degree-1].transform(X_test_mapped)

# Compute the test MSE
yhat = models[degree-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Training MSE: {train_mses[degree-1]:.2f}")
print(f"Cross Validation MSE: {cv_mses[degree-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

## Neural Networks

The same model selection process can also be used when choosing between different neural network architectures. In this section, you will create the models shown below and apply it to the same regression task above.

<img src='images/C2_W3_NN_Arch.png' width=40%>

### เตรียมข้อมูล

คุณจะใช้ชุดการฝึก การตรวจสอบข้าม และชุดทดสอบเดียวกันที่คุณสร้างขึ้นในส่วนก่อนหน้า จากบทเรียนก่อนหน้านี้ในหลักสูตรนี้ คุณอาจรู้ว่าเครือข่ายประสาทเทียมสามารถเรียนรู้ความสัมพันธ์ที่ไม่เชิงเส้นได้ ดังนั้นคุณสามารถเลือกที่จะข้ามการเพิ่มคุณลักษณะพหุนาม โค้ดจะยังคงรวมอยู่ด้านล่างในกรณีที่คุณต้องการลองใช้ในภายหลังและดูว่าจะมีผลต่อผลลัพธ์ของคุณอย่างไร ค่าเริ่มต้นของ  `degree` ถูกตั้งค่าเป็น `1` เพื่อระบุว่าจะใช้ `x_train`, `x_cv`, และ  `x_test`ตามเดิม (เช่น ไม่มีคุณลักษณะพหุนามเพิ่มเติม)

In [ ]:
# Add polynomial features
degree = 1
poly = PolynomialFeatures(degree, include_bias=False)
X_train_mapped = poly.fit_transform(x_train)
X_cv_mapped = poly.transform(x_cv)
X_test_mapped = poly.transform(x_test)

ต่อไป คุณจะปรับขนาดคุณลักษณะอินพุตเพื่อช่วยให้การไล่ระดับลงมาบรรจบกันได้เร็วขึ้น อีกครั้ง โปรดสังเกตว่าคุณกำลังใช้ค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานที่คำนวณจากชุดฝึกโดยใช้  `transform()` ในชุดตรวจสอบข้ามและชุดทดสอบแทนการใช้  `fit_transform()`.

In [ ]:
# Scale the features using the z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train_mapped)
X_cv_mapped_scaled = scaler.transform(X_cv_mapped)
X_test_mapped_scaled = scaler.transform(X_test_mapped)

### สร้างและฝึกอบรมโมเดล


จากนั้นคุณจะสร้างสถาปัตยกรรมเครือข่ายประสาทเทียมที่แสดงไว้ก่อนหน้านี้  โค้ดมีให้ในฟังก์ชัน  `build_models()` ในไฟล์ `utils.py` กรณีที่คุณต้องการตรวจสอบหรือปรับแต่ง คุณจะใช้สิ่งนั้นในลูปด้านล่างจากนั้นดำเนินการฝึกอบรมโมเดล สำหรับแต่ละโมเดล คุณจะบันทึกข้อผิดพลาดในการฝึกอบรมและการตรวจสอบ (training errors & cross validation errors) ด้วย

In [ ]:
# Initialize lists that will contain the errors for each model
nn_train_mses = []
nn_cv_mses = []

# Build the models
nn_models = utils.build_models()

# Loop over the the models
for model in nn_models:
    
    # Setup the loss and optimizer
    model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )

    print(f"Training {model.name}...")
    
    # Train the model
    model.fit(
        X_train_mapped_scaled, y_train,
        epochs=300,
        verbose=0
    )
    
    print("Done!\n")

    
    # Record the training MSEs
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    nn_train_mses.append(train_mse)
    
    # Record the cross validation MSEs 
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    nn_cv_mses.append(cv_mse)

    
# print results
print("RESULTS:")
for model_num in range(len(nn_train_mses)):
    print(
        f"Model {model_num+1}: Training MSE: {nn_train_mses[model_num]:.2f}, " +
        f"CV MSE: {nn_cv_mses[model_num]:.2f}"
        )

จากข้อผิดพลาดที่บันทึกไว้ คุณสามารถตัดสินใจได้ว่าโมเดลใดดีที่สุดสำหรับแอปพลิเคชันของคุณ ดูผลลัพธ์ข้างต้นและดูว่าคุณเห็นด้วยกับ `model_num`  ที่เลือกด้านล่างหรือไม่ สุดท้าย คุณจะคำนวณข้อผิดพลาดในการทดสอบเพื่อประมาณการว่ามันจะสรุปทั่วไปกับตัวอย่างใหม่ได้ดีแค่ไหน

In [ ]:
# Select the model with the lowest CV MSE
model_num = 3

# Compute the test MSE
yhat = nn_models[model_num-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Selected Model: {model_num}")
print(f"Training MSE: {nn_train_mses[model_num-1]:.2f}")
print(f"Cross Validation MSE: {nn_cv_mses[model_num-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

## การจำแนกประเภท

ในส่วนสุดท้ายของห้องปฏิบัติการนี้ คุณจะฝึกฝนการประเมินและเลือกโมเดลในการทำภารกิจการจำแนกประเภท กระบวนการนี้จะคล้ายกัน โดยมีความแตกต่างหลักอยู่ที่การคำนวณข้อผิดพลาด คุณจะเห็นในส่วนต่อไปนี้

### โหลดชุดข้อมูล

ขั้นแรก คุณจะโหลดชุดข้อมูลสำหรับงานการจำแนกประเภทแบบไบนารี มีตัวอย่าง 200 ตัวอย่างของสองคุณลักษณะอินพุต (`x1` และ  `x2`), และเป้าหมาย `y` ที่เป็น `0` หรือ `1`.

In [ ]:
# Load the dataset from a text file
data = np.loadtxt('./data/data_w3_ex2.csv', delimiter=',')

# Split the inputs and outputs into separate arrays
x_bc = data[:,:-1]
y_bc = data[:,-1]

# Convert y into 2-D because the commands later will require it (x is already 2-D)
y_bc = np.expand_dims(y_bc, axis=1)

print(f"the shape of the inputs x is: {x_bc.shape}")
print(f"the shape of the targets y is: {y_bc.shape}")

คุณสามารถพล็อตชุดข้อมูลเพื่อตรวจสอบว่าตัวอย่างถูกแยกออกอย่างไร

In [ ]:
utils.plot_bc_dataset(x=x_bc, y=y_bc, title="x1 vs. x2")

### แบ่งและเตรียมชุดข้อมูล

ต่อไป คุณจะสร้างชุดฝึก ชุดตรวจสอบข้าม และชุดทดสอบ คุณจะใช้สัดส่วน 60/20/20 เดิม คุณจะปรับขนาดคุณลักษณะเช่นเดียวกับที่คุณทำในส่วนก่อนหน้า

In [ ]:
from sklearn.model_selection import train_test_split

# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables.
x_bc_train, x_, y_bc_train, y_ = train_test_split(x_bc, y_bc, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_bc_cv, x_bc_test, y_bc_cv, y_bc_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary variables
del x_, y_

print(f"the shape of the training set (input) is: {x_bc_train.shape}")
print(f"the shape of the training set (target) is: {y_bc_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_bc_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_bc_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_bc_test.shape}")
print(f"the shape of the test set (target) is: {y_bc_test.shape}")

In [ ]:
# Scale the features

# Initialize the class
scaler_linear = StandardScaler()

# Compute the mean and standard deviation of the training set then transform it
x_bc_train_scaled = scaler_linear.fit_transform(x_bc_train)
x_bc_cv_scaled = scaler_linear.transform(x_bc_cv)
x_bc_test_scaled = scaler_linear.transform(x_bc_test)

### การประเมินข้อผิดพลาดสำหรับโมเดลการจำแนกประเภท
ในส่วนก่อนหน้าเกี่ยวกับโมเดลการถดถอย คุณใช้ค่าความคลาดเคลื่อนกำลังสองเฉลี่ย (mean squared error) เพื่อวัดประสิทธิภาพของโมเดลของคุณ สำหรับการจำแนกประเภท คุณสามารถคำนวณตัวชี้วัดที่คล้ายคลึงกันได้โดยการคำนวณสัดส่วนของข้อมูลที่โมเดลจัดประเภทผิดตัวอย่างเช่น สมมติว่าโมเดลของคุณทำนายผิดสำหรับตัวอย่าง 2 จาก 5 ตัวอย่าง คุณจะรายงานข้อผิดพลาดที่  `40%` หรือ `0.4`โค้ดด้านล่างแสดงวิธีการคำนวณนี้โดยใช้ลูป for และฟังก์ชัน [`mean()`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) ของ Numpy

In [ ]:
# Sample model output
probabilities = np.array([0.2, 0.6, 0.7, 0.3, 0.8])

# Apply a threshold to the model output. If greater than 0.5, set to 1. Else 0.
predictions = np.where(probabilities >= 0.5, 1, 0)

# Ground truth labels
ground_truth = np.array([1, 1, 1, 1, 1])

# Initialize counter for misclassified data
misclassified = 0

# Get number of predictions
num_predictions = len(predictions)

# Loop over each prediction
for i in range(num_predictions):
    
    # Check if it matches the ground truth
    if predictions[i] != ground_truth[i]:
        
        # Add one to the counter if the prediction is wrong
        misclassified += 1

# Compute the fraction of the data that the model misclassified
fraction_error = misclassified/num_predictions

print(f"probabilities: {probabilities}")
print(f"predictions with threshold=0.5: {predictions}")
print(f"targets: {ground_truth}")
print(f"fraction of misclassified data (for-loop): {fraction_error}")
print(f"fraction of misclassified data (with np.mean()): {np.mean(predictions != ground_truth)}")

### สร้างและฝึกโมเดล
คุณจะใช้โครงสร้างเครือข่ายประสาทเทียมแบบเดียวกับในหัวข้อก่อนหน้า เพื่อให้คุณสามารถเรียกฟังก์ชัน `build_models()` อีกครั้งเพื่อสร้างโมเดลใหม่เหล่านี้

คุณจะปฏิบัติตามแนวทางที่แนะนำจากสัปดาห์ที่แล้วโดยใช้ฟังก์ชันการกระตุ้นแบบ `linear` สำหรับชั้นเอาต์พุต (แทนที่จะเป็น `sigmoid`) จากนั้นตั้งค่า `from_logits=True` เมื่อประกาศฟังก์ชันการสูญเสียของโมเดล เนื่องจากเป็นปัญหาการจำแนกประเภทแบบไบนารี คุณจะใช้ [binary crossentropy loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) 

หลังจากการฝึก คุณจะใช้ [sigmoid function](https://www.tensorflow.org/api_docs/python/tf/math/sigmoid) เพื่อแปลงผลลัพธ์ของโมเดลเป็นความน่าจะเป็น จากนั้นคุณสามารถตั้งค่าเกณฑ์ (threshold) และรับสัดส่วนของตัวอย่างที่จัดประเภทผิดจากชุดข้อมูลฝึกอบรมและชุดข้อมูลตรวจสอบข้าม

คุณสามารถดูทั้งหมดนี้ได้ในเซลล์โค้ดด้านล่าง

In [ ]:
# Initialize lists that will contain the errors for each model
nn_train_error = []
nn_cv_error = []

# Build the models
models_bc = utils.build_models()

# Loop over each model
for model in models_bc:
    
    # Setup the loss and optimizer
    model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    )

    print(f"Training {model.name}...")

    # Train the model
    model.fit(
        x_bc_train_scaled, y_bc_train,
        epochs=200,
        verbose=0
    )
    
    print("Done!\n")
    
    # Set the threshold for classification
    threshold = 0.5
    
    # Record the fraction of misclassified examples for the training set
    yhat = model.predict(x_bc_train_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    train_error = np.mean(yhat != y_bc_train)
    nn_train_error.append(train_error)

    # Record the fraction of misclassified examples for the cross validation set
    yhat = model.predict(x_bc_cv_scaled)
    yhat = tf.math.sigmoid(yhat)
    yhat = np.where(yhat >= threshold, 1, 0)
    cv_error = np.mean(yhat != y_bc_cv)
    nn_cv_error.append(cv_error)

# Print the result
for model_num in range(len(nn_train_error)):
    print(
        f"Model {model_num+1}: Training Set Classification Error: {nn_train_error[model_num]:.5f}, " +
        f"CV Set Classification Error: {nn_cv_error[model_num]:.5f}"
        )

จากผลลัพธ์ข้างต้น คุณสามารถเลือกได้ว่าอันไหนทำได้ดีที่สุด หากมีการเสมอกันในข้อผิดพลาดของชุดการตรวจสอบข้าม คุณสามารถเพิ่มเกณฑ์อื่นเพื่อตัดสินใจได้ ตัวอย่างเช่น คุณสามารถเลือกอันที่มีข้อผิดพลาดในการฝึกน้อยกว่า วิธีการทั่วไปอีกวิธีหนึ่งคือการเลือกโมเดลที่เล็กกว่าเนื่องจากช่วยประหยัดทรัพยากรการคำนวณ ในตัวอย่างของเรา โมเดล 1 เล็กที่สุดและโมเดล 3 ใหญ่ที่สุด

สุดท้าย คุณสามารถคำนวณข้อผิดพลาดของการทดสอบเพื่อรายงานข้อผิดพลาดในการทั่วไปของโมเดล

In [ ]:
# Select the model with the lowest error
model_num = 3

# Compute the test error
yhat = models_bc[model_num-1].predict(x_bc_test_scaled)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1, 0)
nn_test_error = np.mean(yhat != y_bc_test)

print(f"Selected Model: {model_num}")
print(f"Training Set Classification Error: {nn_train_error[model_num-1]:.4f}")
print(f"CV Set Classification Error: {nn_cv_error[model_num-1]:.4f}")
print(f"Test Set Classification Error: {nn_test_error:.4f}")

## สรุป

ในห้องปฏิบัติการนี้ คุณได้ฝึกฝนการประเมินประสิทธิภาพของโมเดลและการเลือกการกำหนดค่าโมเดลที่แตกต่างกัน คุณได้แยกชุดข้อมูลของคุณออกเป็นชุดฝึก ชุดตรวจสอบข้าม และชุดทดสอบ และได้เห็นวิธีการใช้แต่ละชุดในแอปพลิเคชันการเรียนรู้ของเครื่อง ในส่วนถัดไปของหลักสูตร คุณจะเห็นเคล็ดลับเพิ่มเติมเกี่ยวกับวิธีการปรับปรุงโมเดลของคุณโดยการวินิจฉัยอคติและความแปรปรวน สู้ๆ!